In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import date

In [2]:
url_milanuncios = 'https://www.milanuncios.com/entradas-de-concierto/?s=coldplay%20paris%2017%20pista'
url_viagogo = 'https://www.viagogo.es'

In [3]:
def get_html(url):
	'''
	Devuelve el contenido del html en un objeto beaufitul soup
	'''
	page = requests.get(url)
	return BeautifulSoup(page.content, 'html.parser')

In [4]:
def milanuncios(url):
    '''
    Extrae de la pagina de mil anuncios informacion
    del concierto de Coldpaly el 17 de julio en Paris y 
    devuelve un pandas dataframe
    '''
    anuncios = get_html(url).find_all('div', {'class':'ma-AdCard-detail'})
    ls_titulos, ls_precios, ls_descrip  = [], [], []
    for anuncio in anuncios:
        ls_titulos.append(anuncio.find('h2').get_text())
        ls_precios.append(anuncio.find('span', {'class':'ma-AdPrice-value'}).get_text())
        ls_descrip.append(anuncio.find('p', {'class':'ma-AdCardDescription-text'}).get_text())
    dictionary = {'titulo':ls_titulos, 
                  'descripcion':ls_descrip, 
                  'precio':ls_precios} 
    return pd.DataFrame.from_dict(dictionary)

In [5]:
def viagogo(url):
    #selenium
    driver = webdriver.Safari()
    driver.get(url)
    search = driver.find_element_by_xpath('//*[@id="search"]')
    search.send_keys('coldplay paris 17')
    # espera a cargar y escoge el elemento que sale
    # aqui podria equivocarse y coger otro concierto ? 
    event = WebDriverWait(driver, 4)\
        .until(EC.element_to_be_clickable((By.XPATH, '//*[@id="searchResults"]/div[3]/div/a')))
    event.click()
    slct_qnty = WebDriverWait(driver, 20)\
        .until(EC.element_to_be_clickable((By.XPATH, '//*[@id="qtySelectionModalContent"]/div/div[1]/div[2]/div[1]')))
    slct_qnty.click()
    options = driver.find_elements_by_class_name('dropdown__row')
    for option in options:
        if '2 entradas' in option.text:
            if '12' not in option.text: # si hay 22, 32, 42... no funciona 
                option.click()

    continue_ = driver.find_element_by_xpath('//*[@id="qtySelectionModalContent"]/div/button')
    continue_.click() 
    html = driver.page_source
    driver.close()
    # BS 
    tablon = BeautifulSoup(html, 'html.parser').find('div', {'class':'f-list'}).find_all('div', {'class':'p0'})
    ls_precio = []
    ls_dia = []
    for anuncio in tablon:
        seccion = anuncio.find('span', {'class':'v-title-sml'}).text.strip()
        if 'Pista' in seccion:
            ls_precio.append(anuncio.find('span', {'class':'t-b'}).text.strip())
            ls_dia.append(date.today())
            ls_pagina.append('viagogo')
    dictionary = {'precio':ls_precio, 
                  'pagina':ls_pagina, 
                  'dia':ls_pagina} 
    return pd.DataFrame.from_dict(dictionary)